In [ ]:
import numpy as np
import pandas as pd 
data=pd.read_csv('path_to_calulated_descriptors_csv_file')
data=data.drop(columns=data.columns[0], axis=1)
Y=data["Class"]
X=data.drop(columns=data.columns[-1], axis=1)


In [ ]:
# zerovariance removed

from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(threshold=0)

selector.fit(X)

# Get support (boolean mask of features that are retained)
mask = selector.get_support()

# Filter out features with zero variance
X_filtered = X.loc[:, mask]


In [ ]:
# Standard scaler

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled_1 = scaler.fit_transform(X_filtered)

X_scaled = pd.DataFrame(X_scaled_1, columns=X_filtered.columns)
X_scaled['Label'] = Y

In [ ]:
# Remove_Correlation

correlation_matrix = X_scaled.corr().abs()['Label']

high_corr_mask = correlation_matrix > 0.85

dropped_columns = set()
for col in X_scaled.columns:
    if high_corr_mask[col] and col != 'Label':
        dropped_columns.add(col)
            
X_corr_removed = X_scaled.drop(columns=dropped_columns)
X_corr_unlabelled=X_corr_removed.drop(columns=X_corr_removed.columns[-1], axis=1)


In [ ]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_corr_unlabelled, Y, test_size=0.2, random_state=1)

In [ ]:
X_train['Label']=y_train
X_test['Label']=y_test

In [ ]:
#Boruta

eature_names = X_corr_unlabelled.columns
from sklearn.ensemble import RandomForestClassifier 
from boruta import BorutaPy

# Create BorutaPy instance

# Initialize RandomForestClassifier
forest = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5, n_estimators=40)

# Initialize Boruta
feat_selector = BorutaPy(forest, n_estimators='auto', verbose=3, random_state=1, max_iter=100, perc=90)

# train Boruta
# N.B.: X and y must be numpy arrays
feat_selector.fit(np.array(X_train), np.array(y_train))

# Print support and ranking for each feature
print("\n------Support and Ranking for each feature------")
rank_1_features = []
for i in range(len(feat_selector.support_)):
    print(f"{feature_names[i]} - Rank: {feat_selector.ranking_[i]}", end='')
    if feat_selector.ranking_[i] == 1:
        print(" - Passes the test and has rank 1")
        rank_1_features.append(feature_names[i])
    else:
        print(" - Doesn't pass the rank 1 test")

# Now, rank_1_features contains the names of all features that have rank 1
print("\nFeatures with Rank 1:", rank_1_features)

# To use these rank 1 features for further modeling:
# Convert feature names back to indices to use with pandas DataFrame if necessary
rank_1_indices = [feature_names.tolist().index(f) for f in rank_1_features]

# Selecting rank 1 features for your training and test set
X_train_feat = X_train.iloc[:, rank_1_indices]
X_test_feat = X_test.iloc[:, rank_1_indices]


In [ ]:
X_train_feat['Label']=y_train
X_test_feat['Label']=y_test

In [ ]:
X_train_feat.to_csv('X_train.csv')
X_test_feat.to_csv('X_test.csv')